In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Load pickled data *Eurosat dataset
import pickle
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
import tensorflow as tf
# Reshapes raw array data to numpy array for both pixel data and label
X = np.array(X).reshape(-1, 64, 64, 3)
y = np.array(y)

In [4]:
# Image size based on Eurosat data, for RGB Channel
IMG_SIZE = 64
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [5]:
base_model = tf.keras.applications.resnet.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [6]:
base_model.trainable = False
# Let's take a look at the base model architecture
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [8]:
prediction_layer = tf.keras.layers.Dense(1)

In [9]:
# forming the pretained model from Resnet50 trained on imagenet dataset
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
])

model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.add(tf.keras.layers.Activation('relu'))

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2, 2, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [11]:
len(model.trainable_variables)

2

In [12]:
# Testing with pretained model, now able to obtained above 50% accuracy for unseen data.
# Will continue working on improving the underlying model classification accuracy.
model.fit(X, y, batch_size=32, epochs=1000, validation_split=0.2)

Train on 21600 samples, validate on 5400 samples
Epoch 1/1000
21600/21600 [==============================] - 36s 2ms/sample - loss: 1.2454 - accuracy: 0.6077 - val_loss: 1.8575 - val_accuracy: 0.4498
Epoch 2/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.6005 - accuracy: 0.8090 - val_loss: 1.9383 - val_accuracy: 0.4704
Epoch 3/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.4912 - accuracy: 0.8442 - val_loss: 1.9440 - val_accuracy: 0.4894
Epoch 4/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.4414 - accuracy: 0.8598 - val_loss: 1.9680 - val_accuracy: 0.5006
Epoch 5/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.4111 - accuracy: 0.8681 - val_loss: 2.0201 - val_accuracy: 0.5000
Epoch 6/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.3922 - accuracy: 0.8752 - val_loss: 2.1060 - val_accuracy: 0.4935
Epoch 7/1000
21600/21600 [===================

21600/21600 [==============================] - 22s 1ms/sample - loss: 0.2255 - accuracy: 0.9259 - val_loss: 2.1986 - val_accuracy: 0.5646
Epoch 55/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.2310 - accuracy: 0.9244 - val_loss: 2.2610 - val_accuracy: 0.5576
Epoch 56/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.2118 - accuracy: 0.9280 - val_loss: 2.2511 - val_accuracy: 0.5585
Epoch 57/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.2186 - accuracy: 0.9271 - val_loss: 2.1545 - val_accuracy: 0.5719
Epoch 58/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.2206 - accuracy: 0.9281 - val_loss: 2.2964 - val_accuracy: 0.5559
Epoch 59/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.2156 - accuracy: 0.9291 - val_loss: 2.2844 - val_accuracy: 0.5594
Epoch 60/1000
21600/21600 [==============================] - 22s 1ms/sample - loss: 0.2195 - accuracy:

21600/21600 [==============================] - 21s 983us/sample - loss: 0.2006 - accuracy: 0.9338 - val_loss: 2.6485 - val_accuracy: 0.5426
Epoch 108/1000
21600/21600 [==============================] - 21s 984us/sample - loss: 0.2005 - accuracy: 0.9338 - val_loss: 2.5873 - val_accuracy: 0.5517
Epoch 109/1000
21600/21600 [==============================] - 21s 982us/sample - loss: 0.1940 - accuracy: 0.9359 - val_loss: 2.6674 - val_accuracy: 0.5422
Epoch 110/1000
21600/21600 [==============================] - 21s 985us/sample - loss: 0.2066 - accuracy: 0.9320 - val_loss: 2.6395 - val_accuracy: 0.5459
Epoch 111/1000
21600/21600 [==============================] - 21s 989us/sample - loss: 0.1997 - accuracy: 0.9346 - val_loss: 2.5358 - val_accuracy: 0.5522
Epoch 112/1000
21600/21600 [==============================] - 21s 990us/sample - loss: 0.2067 - accuracy: 0.9309 - val_loss: 2.6415 - val_accuracy: 0.5444
Epoch 113/1000
21600/21600 [==============================] - 21s 986us/sample - loss

Epoch 160/1000
21600/21600 [==============================] - 21s 975us/sample - loss: 0.2027 - accuracy: 0.9342 - val_loss: 2.7286 - val_accuracy: 0.5489
Epoch 161/1000
21600/21600 [==============================] - 21s 975us/sample - loss: 0.1928 - accuracy: 0.9375 - val_loss: 2.7707 - val_accuracy: 0.5446
Epoch 162/1000
21600/21600 [==============================] - 21s 976us/sample - loss: 0.2070 - accuracy: 0.9349 - val_loss: 2.6624 - val_accuracy: 0.5574
Epoch 163/1000
21600/21600 [==============================] - 21s 977us/sample - loss: 0.2007 - accuracy: 0.9392 - val_loss: 2.7845 - val_accuracy: 0.5513
Epoch 164/1000
21600/21600 [==============================] - 21s 977us/sample - loss: 0.1988 - accuracy: 0.9352 - val_loss: 2.7880 - val_accuracy: 0.5467
Epoch 165/1000
21600/21600 [==============================] - 21s 976us/sample - loss: 0.2020 - accuracy: 0.9346 - val_loss: 2.7897 - val_accuracy: 0.5452
Epoch 166/1000
21600/21600 [==============================] - 21s 962u

Epoch 213/1000
21600/21600 [==============================] - 21s 988us/sample - loss: 0.1907 - accuracy: 0.9384 - val_loss: 2.8758 - val_accuracy: 0.5470
Epoch 214/1000
21600/21600 [==============================] - 21s 986us/sample - loss: 0.2088 - accuracy: 0.9334 - val_loss: 2.8534 - val_accuracy: 0.5480
Epoch 215/1000
21600/21600 [==============================] - 21s 982us/sample - loss: 0.1985 - accuracy: 0.9374 - val_loss: 2.9931 - val_accuracy: 0.5348
Epoch 216/1000
21600/21600 [==============================] - 21s 965us/sample - loss: 0.2053 - accuracy: 0.9356 - val_loss: 2.7241 - val_accuracy: 0.5591
Epoch 217/1000
21600/21600 [==============================] - 21s 982us/sample - loss: 0.2028 - accuracy: 0.9359 - val_loss: 2.9301 - val_accuracy: 0.5400
Epoch 218/1000
21600/21600 [==============================] - 21s 984us/sample - loss: 0.1899 - accuracy: 0.9393 - val_loss: 2.8449 - val_accuracy: 0.5498
Epoch 219/1000
21600/21600 [==============================] - 21s 981u

...

Epoch 742/1000
21600/21600 [==============================] - 20s 945us/sample - loss: 0.2188 - accuracy: 0.9388 - val_loss: 3.5906 - val_accuracy: 0.5430
Epoch 743/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2084 - accuracy: 0.9413 - val_loss: 3.7491 - val_accuracy: 0.5335
Epoch 744/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2167 - accuracy: 0.9403 - val_loss: 3.6446 - val_accuracy: 0.5361
Epoch 745/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2161 - accuracy: 0.9397 - val_loss: 3.7012 - val_accuracy: 0.5359
Epoch 746/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2228 - accuracy: 0.9381 - val_loss: 3.7061 - val_accuracy: 0.5350
Epoch 747/1000
21600/21600 [==============================] - 21s 949us/sample - loss: 0.2264 - accuracy: 0.9370 - val_loss: 3.7111 - val_accuracy: 0.5343
Epoch 748/1000
21600/21600 [==============================] - 20s 948u

Epoch 795/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2219 - accuracy: 0.9372 - val_loss: 3.6551 - val_accuracy: 0.5409
Epoch 796/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2154 - accuracy: 0.9403 - val_loss: 3.6679 - val_accuracy: 0.5391
Epoch 797/1000
21600/21600 [==============================] - 20s 945us/sample - loss: 0.2289 - accuracy: 0.9363 - val_loss: 3.6909 - val_accuracy: 0.5357
Epoch 798/1000
21600/21600 [==============================] - 20s 948us/sample - loss: 0.2283 - accuracy: 0.9363 - val_loss: 3.7198 - val_accuracy: 0.5374
Epoch 799/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2211 - accuracy: 0.9387 - val_loss: 3.6148 - val_accuracy: 0.5417
Epoch 800/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2304 - accuracy: 0.9357 - val_loss: 3.7099 - val_accuracy: 0.5331
Epoch 801/1000
21600/21600 [==============================] - 20s 946u

Epoch 848/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2171 - accuracy: 0.9396 - val_loss: 3.6642 - val_accuracy: 0.5448
Epoch 849/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2260 - accuracy: 0.9376 - val_loss: 3.5801 - val_accuracy: 0.5515
Epoch 850/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2201 - accuracy: 0.9383 - val_loss: 3.8315 - val_accuracy: 0.5302
Epoch 851/1000
21600/21600 [==============================] - 20s 945us/sample - loss: 0.2224 - accuracy: 0.9383 - val_loss: 3.7297 - val_accuracy: 0.5389
Epoch 852/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2338 - accuracy: 0.9375 - val_loss: 3.5975 - val_accuracy: 0.5446
Epoch 853/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2301 - accuracy: 0.9370 - val_loss: 3.6971 - val_accuracy: 0.5387
Epoch 854/1000
21600/21600 [==============================] - 20s 946u

Epoch 901/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2240 - accuracy: 0.9396 - val_loss: 3.7501 - val_accuracy: 0.5363
Epoch 902/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2327 - accuracy: 0.9368 - val_loss: 3.7812 - val_accuracy: 0.5374
Epoch 903/1000
21600/21600 [==============================] - 20s 945us/sample - loss: 0.2163 - accuracy: 0.9401 - val_loss: 3.9094 - val_accuracy: 0.5256
Epoch 904/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2147 - accuracy: 0.9414 - val_loss: 3.7994 - val_accuracy: 0.5361
Epoch 905/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2134 - accuracy: 0.9403 - val_loss: 3.9084 - val_accuracy: 0.5269
Epoch 906/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2269 - accuracy: 0.9375 - val_loss: 3.9227 - val_accuracy: 0.5300
Epoch 907/1000
21600/21600 [==============================] - 20s 947u

Epoch 954/1000
21600/21600 [==============================] - 20s 948us/sample - loss: 0.2255 - accuracy: 0.9376 - val_loss: 3.9126 - val_accuracy: 0.5287
Epoch 955/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2367 - accuracy: 0.9364 - val_loss: 3.8693 - val_accuracy: 0.5339
Epoch 956/1000
21600/21600 [==============================] - 20s 945us/sample - loss: 0.2254 - accuracy: 0.9386 - val_loss: 3.8260 - val_accuracy: 0.5350
Epoch 957/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2267 - accuracy: 0.9380 - val_loss: 3.9321 - val_accuracy: 0.5283
Epoch 958/1000
21600/21600 [==============================] - 20s 947us/sample - loss: 0.2301 - accuracy: 0.9386 - val_loss: 3.8760 - val_accuracy: 0.5343
Epoch 959/1000
21600/21600 [==============================] - 20s 946us/sample - loss: 0.2291 - accuracy: 0.9372 - val_loss: 3.9448 - val_accuracy: 0.5294
Epoch 960/1000
21600/21600 [==============================] - 20s 946u

In [13]:
# Able to set tf to compute using local gpu
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1



In [14]:
model.save('models\\first_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models\first_model\assets
